# Mphasis DeepInsights Legal Entity Name Extractor

Mphasis DeepInsights legal entity name extraction is an optimal way to identify and classify legal organization name from their general names in unstructured text. Named entity recognition (NER), is an information extraction technique to identify key elements from text and classifies them into predefined categories. This solution leverages pretrained spacy models to extract organization tags from a given input text.  The input can have a maximum of 50000 characters and gives output as a list of dictionaries containing legal as well as generally pronounced names of any organization.

### Prerequisite

To run this algorithm you need to have access to the following AWS Services:
- Access to AWS SageMaker and the model package.
- An S3 bucket to specify input/output.
- Role for AWS SageMaker to access input/output from S3.

This sample notebook shows you how to deploy Mphasis DeepInsights Legal Entity Name Extraction Sentiment Analyzer using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Mphasis DeepInsights Legal Entity Name Extraction. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Output Result](#D.-Output-Result)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Mphasis DeepInsights Legal Entity Name Extraction
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/legal-entity-v2'

In [2]:
import pandas as pd
import numpy as np
import json
import os
import boto3
from zipfile import ZipFile
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from IPython.display import Image, display

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='legal-entity-1'

content_type='text/plain'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:
def predict_wrapper(endpoint, session):
    return sage.predictor.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.

model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

In [6]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

------!

The class RealTimePredictor has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

#### Instructions

    1) The input has to be a '.txt' file with 'utf-8' encoding.

    2) input file should not contain more than 50000 characters

In [1]:
with open('sample_input.txt', 'rb') as file_stream:
    input_text = file_stream.read().decode('utf-8')

In [2]:
print(input_text)

'Meta Platforms Inc., doing business as Meta and formerly named Facebook Inc., and Facebook Inc., is an American multinational technology conglomerate based in Menlo Park, California. The company owns Facebook, Instagram, and WhatsApp, among other products and services. Meta is one of the world most valuable companies. It is considered one of the Big Five American information technology companies, alongside Alphabet, Amazon, Apple, and Microsoft. Meta products and services include Facebook, Messenger, Facebook Watch, and Meta Portal. It has also acquired Oculus, Giphy, Mapillary, Kustomer, Presize and has a . % stake in Jio Platforms. In , the company generated . % of its revenue from the sale of advertising. In October , the parent company of Facebook changed its name from Facebook Inc., to Meta Platforms Inc., to "reflect its focus on building the metaverse". According to Meta, the "metaverse" refers to the integrated environment that links all of the company products and services. F

In [7]:
file_name = 'sample_input.txt'

#### C. Perform real-time inference

In [8]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$file_name --content-type 'text/plain' --region us-east-2 output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Output Result

- The output file (in json format) contains the following files:

    1. 'output.json': List of dictionary containing extracted Legal names and general names of any organization.


In [9]:
import json
file_path = os.getcwd()
file_name = 'output.json'

#file_object = open(file_name,'rb')
with open(file_name,'r') as f:
    json_array=json.load(f)

In [10]:
json_array

[{'legal_entity': 'apple inc.', 'aka': ['apple', 'apple inc.']},
 {'legal_entity': ['facebook inc.',
   'facebook india online services pvt ltd.',
   'facebook platform cooperative'],
  'aka': ['facebook',
   'facebook india online services pvt ltd.',
   'facebook inc.',
   'facebook platform cooperative',
   'facebook watch']},
 {'legal_entity': 'libra association',
  'aka': ['libra networks', 'libra association']},
 {'legal_entity': 'meta platforms inc.',
  'aka': ['meta',
   'meta portal',
   'meta platforms ireland',
   'meta board',
   'meta platforms inc.']}]

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [11]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
#upload the batch-transform job input files to S3
transform_input_folder = "data/input/batch"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

In [ ]:
#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
#output is available on following path
transformer.output_path

In [ ]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('output.json', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, os.path.basename(transformer.output_path)+'/sample_input.txt.out', f)
    print("Output file loaded from bucket")

In [16]:
import json
file_path = os.getcwd()
file_name = 'output.json'

#file_object = open(file_name,'rb')
with open(file_name,'r') as f:
    json_array=json.load(f)

In [17]:
json_array

[{'legal_entity': 'apple inc.', 'aka': ['apple inc.', 'apple']},
 {'legal_entity': ['facebook inc.',
   'facebook india online services pvt ltd.',
   'facebook platform cooperative'],
  'aka': ['facebook india online services pvt ltd.',
   'facebook inc.',
   'facebook',
   'facebook watch',
   'facebook platform cooperative']},
 {'legal_entity': 'libra association',
  'aka': ['libra networks', 'libra association']},
 {'legal_entity': 'meta platforms inc.',
  'aka': ['meta platforms ireland',
   'meta platforms inc.',
   'meta',
   'meta board',
   'meta portal']}]

### 4. Clean-up

#### A. Delete the model

In [18]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.